In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000503.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000390.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000601.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000030.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000567.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000686.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000042.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000080.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000027.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000127.jpg
/kaggle/input/sports-dataset-for-sports-video-classification/data/ice_hockey/00000113.jpg
/kaggle/in

In [2]:
import tensorflow as tf

2021-10-07 05:49:06.417014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
datapath = r"../input/sports-dataset-for-sports-video-classification/data"
outputmodel = r"../output/kaggle/working/videoclassmodel"
outputlabelbinarizer = r"../output/kaggle/working/videoclassificationbinarizer.pickle"
epoch = 25

In [4]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=ed4b0e9144105f268f00eee7d8dbf2a12f008f4e67deabe8b292c1b6f5c7a8dc
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [5]:
import tensorflow as tf
from tensorflow import keras
import os
from imutils import paths
import numpy as np
import cv2 as cv

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [7]:
sports_labels = set(['boxing','swimming','table_tennis','badminton','basketball','cricket','football','gymnastics','hockey','kabaddi','tennis','volleyball','weight_lifting'])
print("Images is being loaded.....")
pathtoimages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathtoimages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv.imread(images)
    image = cv.cvtColor(image,cv.COLOR_BGR2RGB)
    image = cv.resize(image,(224,224))
    data.append(image)
    labels.append(label)

Images is being loaded.....


In [8]:
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [9]:
X_train,X_test,y_train,y_test = train_test_split(data,labels,test_size=0.25,stratify=labels,random_state=42)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_augment = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validation_augment = ImageDataGenerator()
mean = np.array([123.68,116.779,103.939],dtype="float32")
train_augment.mean = mean
validation_augment.mean = mean

In [12]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.resnet import ResNet50

In [13]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [14]:
basemodel = ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))
headmodel = basemodel.output
headmodel = layers.AveragePooling2D(pool_size=(7,7))(headmodel)
headmodel = layers.Flatten()(headmodel)
headmodel = layers.Dense(512,activation="relu")(headmodel)
headmodel = layers.Dropout(0.5)(headmodel)
headmodel = layers.Dense(len(lb.classes_),activation="softmax")(headmodel)
model = Model(basemodel.input,headmodel)

2021-10-07 05:50:34.200306: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-07 05:50:34.203527: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-07 05:50:34.241006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-07 05:50:34.241634: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-07 05:50:34.241694: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-07 05:50:34.268679: I tensorflow/stream_executor/platform/def

94773248/94765736 [==============================] - 1s 0us/step


In [15]:
for layer in basemodel.layers:
    layer.trainable = False

In [16]:
from tensorflow.keras.optimizers import SGD

In [17]:
class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.98):
            print("Reached 98% accuracy...stoppping training")
            self.model.stop_training = True

In [18]:
callbacks = callback()

In [19]:
model.compile(loss="categorical_crossentropy",optimizer=SGD(lr=0.001,momentum=0.9,decay=1e-4/epoch),metrics=["accuracy"])

In [20]:
History = model.fit_generator(
    train_augment.flow(X_train,y_train,batch_size=32),
    steps_per_epoch = len(X_train)//32,
    validation_data=validation_augment.flow(X_test,y_test),
    validation_steps=len(X_test)//32,
    epochs=epoch,
    callbacks=[callbacks]
)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-10-07 05:50:40.845848: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-07 05:50:40.850688: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000160000 Hz


Epoch 1/25


2021-10-07 05:50:43.533832: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-07 05:50:44.402939: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-07 05:50:44.440832: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


203/203 [==============================] - 81s 359ms/step - loss: 2.3300 - accuracy: 0.3051 - val_loss: 0.9525 - val_accuracy: 0.7174
Epoch 2/25
203/203 [==============================] - 72s 353ms/step - loss: 1.2264 - accuracy: 0.6060 - val_loss: 0.7290 - val_accuracy: 0.7850
Epoch 3/25
203/203 [==============================] - 73s 359ms/step - loss: 1.0212 - accuracy: 0.6700 - val_loss: 0.6421 - val_accuracy: 0.8097
Epoch 4/25
203/203 [==============================] - 73s 358ms/step - loss: 0.8882 - accuracy: 0.7172 - val_loss: 0.6228 - val_accuracy: 0.8144
Epoch 5/25
203/203 [==============================] - 72s 356ms/step - loss: 0.8354 - accuracy: 0.7306 - val_loss: 0.5652 - val_accuracy: 0.8274
Epoch 6/25
203/203 [==============================] - 73s 356ms/step - loss: 0.7819 - accuracy: 0.7446 - val_loss: 0.5517 - val_accuracy: 0.8349
Epoch 7/25
203/203 [==============================] - 72s 355ms/step - loss: 0.7537 - accuracy: 0.7520 - val_loss: 0.5338 - val_accuracy: 0.8

In [21]:
import pickle

In [22]:
# model.save("video_model.h5")
pickle.dump(lb,open('labelpickle.pickle','wb'))

In [23]:
model.save("model.h5")